# 3_training_megatron-lm

- https://github.com/NVIDIA/Megatron-LM 에서 파일을 다운로드 받은 후 3_training_megatron-lm 폴더의 코드를 덮어쓰기하여 학습을 하시면 됩니다.
- 샘플 데이터셋은 https://github.com/huggingface/blog/blob/main/megatron-training.md#data-preprocessing 를 참고해서 만들 수 있습니다. 
- 시작 전 docker 폴더에서 Dockerfile을 이용하여 custom_docker를 만들어야 합니다.

In [1]:
install_needed = True
# install_needed = False

In [ ]:
%%bash
#!/bin/bash

DAEMON_PATH=$PWD/docker"
MEMORY_SIZE=10G

FLAG=$(cat $DAEMON_PATH/daemon.json | jq 'has("data-root")')
#echo $PWD/"3_training_megatron-lm/docker"

if [ "$FLAG" == true ]; then
    echo "Already revised"
else
    echo "Add data-root and default-shm-size=$MEMORY_SIZE"
    sudo cp $DAEMON_PATH/daemon.json $DAEMON_PATH/daemon.json.bak
    sudo cat $DAEMON_PATH/daemon.json.bak | jq '. += {"data-root":"/home/ec2-user/SageMaker/.container/docker","default-shm-size":"'$MEMORY_SIZE'"}' | sudo tee $DAEMON_PATH/daemon.json > /dev/null
    sudo launchctl restart docker
    echo "Docker Restart"
fi

sudo curl -L "https://github.com/docker/compose/releases/download/v2.7.0/docker-compose-$(uname -s)-$(uname -m)" -o /usr/local/bin/docker-compose
sudo chmod +x /usr/local/bin/docker-compose

In [15]:
import sys
import IPython

if install_needed:
    print("installing deps and restarting kernel")
    !{sys.executable} -m pip install --upgrade pip --quiet
    !{sys.executable} -m pip install -U sagemaker --quiet
    IPython.Application.instance().kernel.do_shutdown(True)

installing deps and restarting kernel
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 2.19.1 requires dill<0.3.9,>=0.3.0, but you have dill 0.3.9 which is incompatible.


## Building and registering the container

In [31]:
%%sh

# The name of our algorithm
algorithm_name=pytorch-training

cd 'docker'

#account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
#region=${region:-us-east-1}


# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

#login to public sagemaker repo to get container
aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin 763104351884.dkr.ecr.us-east-1.amazonaws.com
# Get the login command from ECR and execute it directly
aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin 783764619792.dkr.ecr.us-east-1.amazonaws.com

export DOCKER_BUILDKIT=1
# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build -f Dockerfile -t pytorch-training . --platform 'linux/amd64,linux/arm64'
docker tag pytorch-training 783764619792.dkr.ecr.us-east-1.amazonaws.com/pytorch-training:latest
docker push 783764619792.dkr.ecr.us-east-1.amazonaws.com/pytorch-training:latest

Login Succeeded
Login Succeeded


#0 building with "desktop-linux" instance using docker driver

#1 [internal] load build definition from Dockerfile
#1 transferring dockerfile: 1.54kB done
#1 DONE 0.0s

#2 [auth] sharing credentials for 763104351884.dkr.ecr.us-east-1.amazonaws.com
#2 DONE 0.0s

#3 [linux/arm64 internal] load metadata for 763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-training:2.3.0-gpu-py311-cu121-ubuntu20.04-sagemaker
#3 ...

#4 [linux/amd64 internal] load metadata for 763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-training:2.3.0-gpu-py311-cu121-ubuntu20.04-sagemaker
#4 DONE 1.0s

#5 [internal] load .dockerignore
#5 transferring context: 2B done
#5 DONE 0.0s

#3 [linux/arm64 internal] load metadata for 763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-training:2.3.0-gpu-py311-cu121-ubuntu20.04-sagemaker
#3 DONE 1.4s

#6 [linux/amd64 1/6] FROM 763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-training:2.3.0-gpu-py311-cu121-ubuntu20.04-sagemaker@sha256:981b1ce0d1099ffba1f8842b7632c6941

The push refers to repository [783764619792.dkr.ecr.us-east-1.amazonaws.com/pytorch-training]
5bc37799e39a: Waiting
3534b7fdf5f7: Waiting
24ff69e0a1e4: Waiting
596fc4dee4de: Waiting
974568495310: Waiting
1b24adeb3c1b: Waiting
ddf22c091a66: Waiting
ee8d87462526: Waiting
813af4514c4b: Waiting
2518b52d49b6: Waiting
9fc2b73e5eeb: Waiting
76a627ca5e65: Waiting
ae33dbb3a55f: Waiting
aa3219d64781: Waiting
03dd81e371d6: Waiting
355e2d5e2f80: Waiting
eb7bf46d7693: Waiting
c696bac94bf0: Waiting
23c4f519318b: Waiting
112feee19fa1: Waiting
7ccec220b45e: Waiting
f146897276ef: Waiting
15d7ad785fe7: Waiting
c79e9e16e5ac: Waiting
3805d1ba40b2: Waiting
2e1d0bd18d40: Waiting
7a2c55901189: Waiting
9d193d3dd589: Waiting
67a3cf0e06d5: Waiting
ac82fa248a4e: Waiting
b4f9cb2f5eba: Waiting
a0fe6398ac18: Waiting
46db5d6dd998: Waiting
359b8c3abf87: Waiting
59c5e12f7b5e: Waiting
e2c3b6c30e8d: Waiting
7faa1993c119: Waiting
387d044338c6: Waiting
c003784c15fd: Waiting
5d14d2bf9f54: Waiting
755e535b54a3: Waiting
3581

## Import Setting

In [32]:
import sagemaker
from pathlib import Path
from time import strftime

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/megatron-lm'

#role = sagemaker.get_execution_role()
role = "arn:aws:iam::783764619792:role/Sagemaker_execution_role"

In [33]:
sagemaker.__version__

'2.240.0'

#### Uploading train data

In [ ]:
!python Megatron-LM/tools/preprocess_data.py \
       --input codeparrot_data.json \
       --output-prefix tiktok_codeparrot \
       --tokenizer-type TikTokenizer \
       --tiktoken-pattern v2 \
       --tokenizer-model list_vocab.json \
       --merge-file merges.txt \
       --json-keys content \
       --workers 32 \
       --append-eod

In [6]:
import glob
import boto3
import os

BUCKET_NAME = 'code-parrot-sample-data'

session = boto3.Session()
s3 = session.client('s3')

filename = os.getcwd()+"/codeparrot_content_document.bin"
key = "qwen_codeparrot_content_document.bin"
print("Putting %s as %s" % (filename,key))
s3.upload_file(filename, BUCKET_NAME, key)

filename = os.getcwd()+"/codeparrot_content_document.idx"
key = "qwen_codeparrot_content_document.idx"
print("Putting %s as %s" % (filename,key))
s3.upload_file(filename, BUCKET_NAME, key)

print("All_Done")

Putting /Users/minsup/Documents/lion/Megatron/codeparrot_content_document.bin as qwen_codeparrot_content_document.bin
Putting /Users/minsup/Documents/lion/Megatron/codeparrot_content_document.idx as qwen_codeparrot_content_document.idx
All_Done


In [16]:
import os
import boto3
from tqdm import tqdm

BUCKET_NAME = 'code-parrot-sample-data'
def upload_folder_to_s3(local_folder, bucket_name, s3_folder=""):
    """
    Uploads a folder and all its contents to an S3 bucket with a progress bar.
    
    :param local_folder: Path to the local folder to upload.
    :param bucket_name: The target S3 bucket name.
    :param s3_folder: The target folder in S3 (optional).
    """
    s3_client = boto3.client('s3')

    # Collect all file paths
    file_list = []
    for root, _, files in os.walk(local_folder):
        for file in files:
            local_file_path = os.path.join(root, file)
            file_list.append(local_file_path)

    # Upload files with progress bar
    with tqdm(total=len(file_list), desc="Uploading", unit="file") as pbar:
        for local_file_path in file_list:
            relative_path = os.path.relpath(local_file_path, local_folder)
            s3_path = os.path.join(s3_folder, relative_path) if s3_folder else relative_path
            s3_path = s3_path.replace("\\", "/")  # Ensure S3 compatibility

            s3_client.upload_file(local_file_path, bucket_name, s3_path)
            pbar.update(1)  # Update progress bar
            print(f"Uploaded: {local_file_path} -> s3://{bucket_name}/{s3_path}")

# Example Usage
upload_folder_to_s3("qwen2.5-7B", BUCKET_NAME, "qwen2.5-7B")

Uploading:   2%|▋                           | 1/43 [06:23<4:28:19, 383.32s/file]

Uploaded: qwen2.5-7B/model-00004-of-00004.safetensors -> s3://code-parrot-sample-data/model-00004-of-00004.safetensors


Uploading:   5%|█▎                          | 2/43 [13:04<4:29:10, 393.91s/file]

Uploaded: qwen2.5-7B/model-00001-of-00004.safetensors -> s3://code-parrot-sample-data/model-00001-of-00004.safetensors


Uploading:   7%|█▉                          | 3/43 [13:04<2:22:47, 214.20s/file]

...

Uploaded: qwen2.5-7B/.cache/huggingface/download/README.md.metadata -> s3://code-parrot-sample-data/.cache/huggingface/download/README.md.metadata


Uploading:  98%|█████████████████████████████▎| 42/43 [26:44<00:00,  2.84file/s]

Uploaded: qwen2.5-7B/.cache/huggingface/download/README.md.lock -> s3://code-parrot-sample-data/.cache/huggingface/download/README.md.lock


Uploading: 100%|██████████████████████████████| 43/43 [26:45<00:00, 37.33s/file]

Uploaded: qwen2.5-7B/.cache/huggingface/download/model-00001-of-00004.safetensors.lock -> s3://code-parrot-sample-data/.cache/huggingface/download/model-00001-of-00004.safetensors.lock


#### Downloading Tokenizer Model

In [539]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-7B")
tokenizer.save_pretrained("./qwen_tokenizer")

/opt/anaconda3/envs/genai/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/anaconda3/envs/genai/lib/python3.12/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


('./qwen_tokenizer/tokenizer_config.json',
 './qwen_tokenizer/special_tokens_map.json',
 './qwen_tokenizer/vocab.json',
 './qwen_tokenizer/merges.txt',
 './qwen_tokenizer/added_tokens.json',
 './qwen_tokenizer/tokenizer.json')

In [540]:
upload_folder_to_s3("qwen_tokenizer", BUCKET_NAME, "megatron-qwen2.5-7B/release/mp_rank_00/")

Uploading:  17%|█████                         | 1/6 [00:00<00:04,  1.19file/s]

Uploaded: qwen_tokenizer/added_tokens.json -> s3://code-parrot-sample-data/megatron-qwen2.5-7B/release/mp_rank_00/added_tokens.json


Uploading:  33%|██████████                    | 2/6 [00:01<00:02,  1.54file/s]

Uploaded: qwen_tokenizer/tokenizer_config.json -> s3://code-parrot-sample-data/megatron-qwen2.5-7B/release/mp_rank_00/tokenizer_config.json


Uploading:  50%|███████████████               | 3/6 [00:01<00:01,  1.94file/s]

...

Uploaded: qwen_tokenizer/tokenizer.json -> s3://code-parrot-sample-data/megatron-qwen2.5-7B/release/mp_rank_00/tokenizer.json


Uploading:  83%|█████████████████████████     | 5/6 [00:12<00:03,  3.05s/file]

Uploaded: qwen_tokenizer/merges.txt -> s3://code-parrot-sample-data/megatron-qwen2.5-7B/release/mp_rank_00/merges.txt


Uploading: 100%|██████████████████████████████| 6/6 [00:13<00:00,  2.29s/file]

Uploaded: qwen_tokenizer/vocab.json -> s3://code-parrot-sample-data/megatron-qwen2.5-7B/release/mp_rank_00/vocab.json


In [34]:
filename = os.path.dirname(os.getcwd())+"/vocab.json"
key = "vocab.json"
print("Putting %s as %s" % (filename,key))
s3.upload_file(filename, BUCKET_NAME, key)

filename = os.path.dirname(os.getcwd())+"/merges.txt"
key = "merges.txt"
print("Putting %s as %s" % (filename,key))
s3.upload_file(filename, BUCKET_NAME, key)

Putting /Users/minsup/Documents/lion/vocab.json as vocab.json
Putting /Users/minsup/Documents/lion/merges.txt as merges.txt


In [34]:
# Configure FSx Input for your SageMaker Training job
from sagemaker.inputs import FileSystemInput

file_system_directory_path= '/wjk6tb4v' #
file_system_id='fs-093b1c9f0849de55a'         # 
file_system_access_mode='rw'
file_system_type='FSxLustre'
train_fs=FileSystemInput(file_system_id=file_system_id,
                         file_system_type=file_system_type,
                         directory_path=file_system_directory_path,
                         file_system_access_mode=file_system_access_mode)

In [7]:
convert_hp = {
    #"load": "/opt/ml/input/data/dataset/dataset/qwen2.5-7B",  # Source Hugging Face model path
    "load": "Qwen/Qwen2.5-7B",
    "save": "/opt/ml/code/qwen2.5-7B-mcore",  # Target Megatron-Core model path
    "tensor-model-parallel-size": 1,  # Tensor Parallelism
    "pipeline-model-parallel-size": 1,  # Pipeline Parallelism
    "micro-batch-size": 1,
    "save-interval": 1,
    "swiglu": None,  # Flags don't take values, use None
    "num-layers": 28,
    "hidden-size": 3584,
    "ffn-hidden-size": 18944,
    "num-attention-heads": 28,
    "max-position-embeddings": 131072,
    "seq-length": 1,
    "patch-tokenizer-type": "Qwen2Tokenizer",
    "extra-vocab-size": 421,
    "normalization": "RMSNorm",
    "attention-dropout": 0.0,
    "hidden-dropout": 0.0,
    "rotary-base": 1000000,
    "transformer-impl": "transformer_engine",  # Transformer Engine support
    "bf16": None,  # Flags should not have values
    "intermediate-size": 18944,
    "group-query-attention": None,  # Flags should not have values
    "norm-epsilon": 1e-6,
    "num-query-groups": 4,
    "untie-embeddings-and-output-weights": None,  # Flags should not have values
}

### Model training with Distributed Data Parallel


The training script provides the code you need for distributed data parallel (DDP) training. The training script is very similar to a PyTorch training script you might run outside of SageMaker.

In the following code block, you can update the estimator function to use a different instance type, instance count, and distrubtion strategy. You're also passing in the training script you reviewed in the previous cell.

In [35]:
#hyperparameters for qwen 7B
hyperparameters = {
    # 'huggingface_model_name_or_path':'Qwen/Qwen2.5-7B',
    # #'language-model':'Qwen/Qwen2.5-7B',
    'load': '/opt/ml/input/data/dataset/dataset/megatron-qwen2.5-7B/release/mp_rank_00/',
    'patch-tokenizer-type': 'Qwen2Tokenizer',
    "hidden-size":3584,
    "num-layers":28,
    "seq-length": 4096, #determines the maximum length of sequences that your model can process in one forward pass (watch OOM)
    "intermediate-size":18944,
    "max-position-embeddings":131072,
    "num-query-groups": 4,
    "group-query-attention":None,
    "untie-embeddings-and-output-weights":None,
    "num-attention-heads":28,
    "norm-epsilon":1e-6,
    "extra-vocab-size":421,
    'micro-batch-size': 1,
    'global-batch-size': 64,
    'lr': 0.0005,
    'train-iters': 100000,
    'lr-decay-iters': 10000,
    'lr-decay-style': 'cosine',
    'lr-warmup-iters': 1000,
    'bf16' : None,
    'log-interval': 10,
    'save-interval': 2000,
    'eval-interval': 200,
    'eval-iters': 10,
    'data-path':'/opt/ml/input/data/dataset/dataset/qwen_codeparrot_content_document',
    'vocab-file':'/opt/ml/input/data/dataset/dataset/vocab.json',
    'merge-file':'/opt/ml/input/data/dataset/dataset/merges.txt',
    'save' : '/opt/ml/model/',
    'tensor-model-parallel-size' : 2,
    'pipeline-model-parallel-size' : 1,
    'context-parallel-size' : 1,
    'weight-decay': 0.1,
    'adam-beta1': 0.9,
    'adam-beta2': 0.95,
    'clip-grad': 1.0,
    'init-method-std': 0.008,
    'attention-dropout': 0.0,
    'hidden-dropout': 0.0,
    'log-throughput': None,
    'no-load-optim': None,
    'no-load-rng': None,
    'num-workers': 8,
    'use-rotary-position-embeddings': None,
    'position-embedding-type': 'rope',
    'disable-bias-linear': None,
    'add-qkv-bias': None,
    'rotary-percent': 1.0,
    'rotary-base': 1000000,
    'rotary-seq-len-interpolation-factor': 1,
    'no-save-optim': None,
    'train-mode':'pretrain',
    'log-progress':None,
    
    #'distributed-timeout-minutes': 60,
    'use-flash-attn': None,  # Enable Flash Attention for better performance
    #'use-ring-exchange-p2p': None,  # More efficient P2P communication
    #'tp-comm-overlap-rs-dgrad': None,  # Enable overlap for reduce-scatter in backward pass
    #'use-cpu-initialization': None,
    
    #'data-cache-path': '/opt/ml/input/data/cache'
}

In [36]:
# TP_COMM_OVERLAP = 1 if hyperparameters['tensor-model-parallel-size'] > 1 else 0
# if TP_COMM_OVERLAP:
#     hyperparameters['tp-comm-overlap']=None
#     hyperparameters['overlap-grad-reduce']=None
#     hyperparameters['overlap-param-gather']=None
# else:
#     hyperparameters['overlap-grad-reduce']=None
#     hyperparameters['overlap-param-gather']=None

# Precision options (based on bf16 setting)
if 'bf16' not in hyperparameters.keys():
    hyperparameters.update({
        'fp16': None,
        'apply-query-key-layer-scaling': None
    })
    hyperparameters["nvte-apply-qk-layer-scaling"] = 1

# Sequence Parallel options (if TP > 1)
if hyperparameters['tensor-model-parallel-size'] > 1:
    hyperparameters['sequence-parallel'] = None

# Dataset options (assuming MMAP type)       
hyperparameters.update({
    'split': '99,1,0',
    'dataset': 'MMAP'
})

# #Activation Checkpointing
# hyperparameters.update({
#     'recompute-method': 'uniform',
#     'recompute-num-layers': 4,  # Default is 1
#     'recompute-granularity': 'full'
# })

# # Add CPU offloading
# hyperparameters.update({
#     'cpu-offloading': True,
#     'cpu-offloading-num-layers': 4
# })

# Make sure optimizer settings are correct
hyperparameters.update({
    'use-distributed-optimizer': None,
    'optimizer': 'hybridadam',
    'optimizer-offload-policy': 'static',
    'optimizer-offload-fraction': 1.0,
})

In [37]:
distribution = {}
distribution = {
    "mpi": {
        "enabled": True
    }
}



## `Pytorch` estimator를 이용한 training job 생성하기


<p><strong><code>sagemaker.pytorch.PyTorch</code></strong> estimator는 처음 실행하는 스크립트 위치와 다양한 연계 코드들이 위치한 디렉토리 정보를 찾아서 스크립트를 S3에 upload하고 SageMaker의 training job을 수행하게 됩니다. training job은 학습을 수행한 단위입니다. 학습을 1번 돌리면 training job이 1개 생성됩니다. 몇 가지 중요 파라미터를 아래와 같이 설명드립니다. </p>

- **entry_point** : 학습을 처음 실행하는 Python 소스 파일의 절대 또는 상대 경로이며, source_dir이 지정된 경우 entry_point는 source_dir 내 파일이 됩니다.
- **source_dir** : 학습에 연계되는 다양한 소스코드 파일이 들어 있는 디렉토리 위치이며, 절대, 상대 경로 또는 S3 URI가 모두 가능하며,source_dir이 S3 URI 인 경우 tar.gz 파일이 됩니다.
- **role** : Amazon SageMaker가 사용자를 대신해 작업(예: S3 버킷에서 모델 결과물이라고 하는 훈련 결과 읽기 및 Amazon S3에 훈련 결과 쓰기)을 수행하는 AWS Identity and Access Management(IAM) 역할입니다.
- **train_instance_count** : 학습을 수행하는 instance 개수를 정의할 수 있습니다.
- **train_instance_type** : 학습을 수행하는 instance 타입을 정의할 수 있습니다.
- **train_volume_size** : 학습 인스턴스에 연결할 Amazon Elastic Block Store(Amazon EBS) 스토리지 볼륨의 크기(GB)입니다. File 모드를 사용할 경우 이 값이 훈련 데이터를 충분히 저장할 수 있는 크기여야 합니다(File 모드가 기본값)
- **train_max_run** : 최대 학습 시간을 설정할 수 있으며, 이 시간이 지나면 Amazon SageMaker는 현재 상태에 관계없이 작업을 종료합니다. (기본값 : 24 * 60 * 60)
- **framework_version** : 학습에 사용될 특정 Pytorch 버전을 정의할 수 있습니다.
- **py_version** : 컨테이너 환경이 python3일 경우 py3, python2일 경우 py2로 설정하면 됩니다. python2는 지원이 중단되었지만 기존 python2로 구성된 파일들을 지원하기 위해 현재 계속 사용할 수 있습니다. 없을 경우에는 기본적으로 py3 입니다.
- **hyperparameters** : 학습에 사용할 하이퍼 파라미터를 정의할 수 있으며, 정의된 하이퍼 파라미터 값들은 모두 학습 컨테이너로 전송이 됩니다.
- **distribution** : 분산과 관련된 값들을 학습 컨테이너로 전송합니다.

<p> 추가적으로 분산/ 멀티 GPU 학습도 가능합니다. SageMaker는 <strong><a href="https://github.com/horovod/horovod" target="_blank" class ='btn-default'>Horovod</a></strong>에 최적화된 환경을 제공하고 있으며, Pytorch의 경우 1.5.0부터 기본 docker에서 apex를 지원합니다.</p>


In [38]:
account_id = "783764619792"
region_name = "us-east-1"
image_uri = f'{account_id}.dkr.ecr.{region_name}.amazonaws.com/pytorch-training:latest'
image_uri

'783764619792.dkr.ecr.us-east-1.amazonaws.com/pytorch-training:latest'

In [39]:
env = {
    'CUDA_DEVICE_MAX_CONNECTIONS':'1'
}

In [40]:
# instance_type = 'ml.p3.16xlarge'  # 'ml.p3.16xlarge', 'ml.p3dn.24xlarge', 'ml.p4d.24xlarge', 'local_gpu'
#instance_type = 'ml.p4d.24xlarge'
instance_type = 'ml.g5.48xlarge'
#instance_type = 'local_gpu'
instance_count = 2
max_run = 4*60*60

In [41]:
if instance_type =='local_gpu':
    from sagemaker.local import LocalSession

    sagemaker_session = LocalSession()
    sagemaker_session.config = {'local': {'local_code': True}}
    s3_data_path = f'file://{Path.cwd().parent}/codeparrot'
    instance_count = 1
else:
    sagemaker_session = sagemaker.Session()
    #s3_data_path = "s3://bucket-name-XXXXXXXX/megatron-lm/codeparrot/"
    s3_data_path = train_fs
s3_data_path

In [60]:
from sagemaker.pytorch import PyTorch
from sagemaker.network import NetworkConfig
from pathlib import Path

estimator = PyTorch(
                    entry_point='debug.py',
                    source_dir=f'{Path.cwd()}',
                    role=role,
                    image_uri=image_uri,
                    framework_version='2.3.0',
                    py_version='py311',
                    instance_count=instance_count,
                    instance_type=instance_type,
                    distribution=distribution,
                    disable_profiler=True,
                    debugger_hook_config=False,
                    max_run=max_run,
                    hyperparameters=hyperparameters,
                    sagemaker_session=sagemaker_session,
                    subnets=['subnet-0bacb66e54e14ffe8'],
                    security_group_ids=['sg-006609bebf68f39d8'],
                    environment=env
                   )

After we've constructed our `PyTorch` object, we can fit it using the data we uploaded to S3. SageMaker makes sure our data is available in the local filesystem, so our training script can simply read the data from disk.


In [61]:
current_time = strftime("%m%d-%IH%MM%Ssec%p")
i_type = instance_type.replace('.','-')
job_name = f'megatron-lm-{i_type}-{current_time}'

estimator.fit(
    inputs={'dataset': s3_data_path}, 
    job_name=job_name,
    wait=False
)

[03/17/25 16:12:31] INFO     SageMaker Python SDK will collect telemetry to help us better  ]8;id=309250;file:///opt/anaconda3/envs/genai/lib/python3.12/site-packages/sagemaker/telemetry/telemetry_logging.py\telemetry_logging.py]8;;\:]8;id=703425;file:///opt/anaconda3/envs/genai/lib/python3.12/site-packages/sagemaker/telemetry/telemetry_logging.py#91\91]8;;\
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

[03/17/25 16:13:10] INFO     Creating training-job with name:                                       ]8;id=55335;file:///opt/anaconda3/envs/genai/lib/python3.12/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=409490;file:///opt/anaconda3/envs/genai/lib/python3.12/site-packages/sagemaker/session.py#1042\1042]8;;\
                             megatron-lm-ml-g5-48xlarge-0317-04H12M31secPM                                         

In [62]:
sagemaker_session = sagemaker.Session()
sagemaker_session.logs_for_job(job_name=job_name, wait=True)

2025-03-17 07:13:11 Starting - Starting the training job...
.........7 07:13:19 Pending - Training job waiting for capacity
.........7 07:15:24 Pending - Preparing the instances for training
...............6:55 Downloading - Downloading the training image
....bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
CUDA compat package should be installed for NVIDIA driver smaller than 530.30.02
Current installed NVIDIA driver version is 550.144.03
Skipping CUDA compat setup as newer NVIDIA driver is installed
/opt/conda/lib/python3.11/site-packages/paramiko/pkey.py:100: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from cryptography.hazmat.primitives.ciphers.algorithms in 48.0.0.
  "cipher": algorithms.TripleDES,
/opt/conda/lib/python3.11/site-packages/paramiko/transport.py:259: CryptographyDeprecationWarning: TripleDES has been moved to cry